#### Note

In [1]:
# pd.set_option('display.max_rows', 100) # 최대 display option set
# pd.reset_option('display.max_rows') # 최대 display option reset

# from datetime import timedelta
# datetime.datetime(2019,12,31) - datetime.datetime(2008,1,1)

#### IMPORT

In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

### IMPORT DATA

In [2]:
asthma = pd.read_csv('outcome/asthma_sudogwon_agg.csv', encoding = 'euc-kr')
rhinitis = pd.read_csv('outcome/rhinitis_sudogwon_agg.csv', encoding = 'euc-kr')
atopic = pd.read_csv('outcome/atopic_sudogwon_agg.csv', encoding = 'euc-kr')
air_out_idx = asthma['air_out_idx'].unique().tolist()

In [3]:
ses = pd.read_excel('SES//administrative_data_air_out_idx_re.xlsx', sheet_name="데이터_정리")
ses.columns = ses.iloc[0]
ses = ses[1:]

In [52]:
# select 2018 only

ses_2018 = ses.loc[ses['요양기관 수 년도'] == 2018].reset_index(drop = True)
ses_2018

,ADMIN_NAME,ADMIN_NAME_2,MAP_NAME,air_out_idx,임시칼럼,인구 년도,소계,도시인구(B),비도시인구(C=A-B),도시인구(D),농촌인구(E=A-D),용도지역 인구기준(B/A*100),행정구역 인구기준(D/A*100),요양기관 수 년도,요양기관 수,재정자립도 년도,재정자립도
0,가평군,경기도 가평군,가평군,1,NaN,2018,62918,36726,26192,19749,43169,58.37,31.39,2018,113,2018,19.7
1,강남구,서울특별시 강남구,강남구,2,NaN,2018,542364,542364,0,542364,0,100,100,2018,3019,2018,53.3
2,강동구,서울특별시 강동구,강동구,3,NaN,2018,427573,427573,0,427573,0,100,100,2018,998,2018,31
3,강릉시,강원도 강릉시,강릉시,4,NaN,2018,212957,185570,27387,184098,28859,87.14,86.45,2018,342,2018,18.2
4,강북구,서울특별시 강북구,강북구,5,NaN,2018,319164,319164,0,319164,0,100,100,2018,614,2018,17.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,화천군,강원도 화천군,화천군,249,NaN,2018,25084,13821,11263,8614,16470,55.1,34.34,2018,36,2018,7.5
248,횡성군,강원도 횡성군,횡성군,250,NaN,2018,46726,21222,25504,21070,25656,45.42,45.09,2018,69,2018,10
249,흥덕구,충청북도 흥덕구,청주시 흥덕구,224,충청북도 청주시,2018,837749,763339,74410,771888,65861,91.12,92.14,2018,1396,2018,33.4
250,제주특별자치도,제주특별자치도 제주시,NaN,NaN,제주특별자치도 본청,2018,485946,447848,38098,473201,12745,92.16,97.38,2018,895,2018,34.5


In [15]:
len(ses_2018['air_out_idx'].dropna().unique())

240

In [16]:
len(air_out_idx)

72

In [27]:
# select only sudogwon data

ses_2018_sudogwon = ses_2018.loc[ses_2018['air_out_idx'].isin(air_out_idx) == True].reset_index(drop = True)
ses_2018_sudogwon

,ADMIN_NAME,ADMIN_NAME_2,MAP_NAME,air_out_idx,임시칼럼,인구 년도,소계,도시인구(B),비도시인구(C=A-B),도시인구(D),농촌인구(E=A-D),용도지역 인구기준(B/A*100),행정구역 인구기준(D/A*100),요양기관 수 년도,요양기관 수,재정자립도 년도,재정자립도
0,가평군,경기도 가평군,가평군,1,NaN,2018,62918,36726,26192,19749,43169,58.37,31.39,2018,113,2018,19.7
1,강남구,서울특별시 강남구,강남구,2,NaN,2018,542364,542364,0,542364,0,100,100,2018,3019,2018,53.3
2,강동구,서울특별시 강동구,강동구,3,NaN,2018,427573,427573,0,427573,0,100,100,2018,998,2018,31
3,강북구,서울특별시 강북구,강북구,5,NaN,2018,319164,319164,0,319164,0,100,100,2018,614,2018,17.6
4,강서구,서울특별시 강서구,강서구,6,NaN,2018,596949,596949,0,596949,0,100,100,2018,1031,2018,21.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,팔달구,경기도 팔달구,수원시 팔달구,128,경기도 수원시,2018,1201166,1201166,0,1201166,0,100,100,2018,2023,2018,49.3
68,평택시,경기도 평택시,평택시,233,NaN,2018,495642,463425,32217,456691,38951,93.5,92.14,2018,717,2018,45.6
69,포천시,경기도 포천시,포천시,234,NaN,2018,150676,80777,69899,78609,72067,53.61,52.17,2018,205,2018,27.3
70,하남시,경기도 하남시,하남시,237,NaN,2018,254415,254415,0,254415,0,100,100,2018,353,2018,44.1


In [28]:
ses_2018_sudogwon.drop(['ADMIN_NAME', 'ADMIN_NAME_2', 'MAP_NAME', '임시칼럼', '인구 년도 ', '요양기관 수 년도', '재정자립도 년도 '], axis = 1, inplace = True)
ses_2018_sudogwon

,air_out_idx,소계,도시인구(B),비도시인구(C=A-B),도시인구(D),농촌인구(E=A-D),용도지역 인구기준(B/A*100),행정구역 인구기준(D/A*100),요양기관 수,재정자립도
0,1,62918,36726,26192,19749,43169,58.37,31.39,113,19.7
1,2,542364,542364,0,542364,0,100,100,3019,53.3
2,3,427573,427573,0,427573,0,100,100,998,31
3,5,319164,319164,0,319164,0,100,100,614,17.6
4,6,596949,596949,0,596949,0,100,100,1031,21.5
...,...,...,...,...,...,...,...,...,...,...
67,128,1201166,1201166,0,1201166,0,100,100,2023,49.3
68,233,495642,463425,32217,456691,38951,93.5,92.14,717,45.6
69,234,150676,80777,69899,78609,72067,53.61,52.17,205,27.3
70,237,254415,254415,0,254415,0,100,100,353,44.1


In [29]:
ses_2018_sudogwon.rename(columns = {'소계' : '전체인구', '도시인구(B)' : '도시인구(용도지역기준)', '비도시인구(C=A-B)': '비도시인구(용도지역기준)', 
                                    '도시인구(D)' : '도시인구(행정구역기준)', '농촌인구(E=A-D)' : '농촌인구(행정구역기준)', 
                                    '용도지역 인구기준(B/A*100)' : '도시지역인구비율(용도지역기준)', '행정구역 인구기준(D/A*100)' : '도시지역인구비율(행정구역기준)'}
                                    , inplace = True)
ses_2018_sudogwon

,air_out_idx,전체인구,도시인구(용도지역기준),비도시인구(용도지역기준),도시인구(행정구역기준),농촌인구(행정구역기준),도시지역인구비율(용도지역기준),도시지역인구비율(행정구역기준),요양기관 수,재정자립도
0,1,62918,36726,26192,19749,43169,58.37,31.39,113,19.7
1,2,542364,542364,0,542364,0,100,100,3019,53.3
2,3,427573,427573,0,427573,0,100,100,998,31
3,5,319164,319164,0,319164,0,100,100,614,17.6
4,6,596949,596949,0,596949,0,100,100,1031,21.5
...,...,...,...,...,...,...,...,...,...,...
67,128,1201166,1201166,0,1201166,0,100,100,2023,49.3
68,233,495642,463425,32217,456691,38951,93.5,92.14,717,45.6
69,234,150676,80777,69899,78609,72067,53.61,52.17,205,27.3
70,237,254415,254415,0,254415,0,100,100,353,44.1


In [37]:
# merge with main data

asthma_sudogwon_agg_2018SESadded = pd.merge(asthma, ses_2018_sudogwon, on='air_out_idx', how = 'left')
rhinitis_sudogwon_agg_2018SESadded = pd.merge(rhinitis, ses_2018_sudogwon, on='air_out_idx', how = 'left')
atopic_sudogwon_agg_2018SESadded = pd.merge(atopic, ses_2018_sudogwon, on='air_out_idx', how = 'left')

In [38]:
asthma_sudogwon_agg_2018SESadded.to_csv('outcome/asthma_sudogwon_agg_2018SESadded.csv', encoding = 'euc-kr', index = False)
rhinitis_sudogwon_agg_2018SESadded.to_csv('outcome/rhinitis_sudogwon_agg_2018SESadded.csv', encoding = 'euc-kr', index = False)
atopic_sudogwon_agg_2018SESadded.to_csv('outcome/atopic_sudogwon_agg_2018SESadded.csv', encoding = 'euc-kr', index = False)

### 2015 data

In [7]:
# 2015 데이터 추가

hospital = pd.read_csv('ses/시군구별_종별_요양기관_현황_20220916152410.csv', encoding = 'cp949')
money = pd.read_csv('ses/재정자립도_시도_시_군_구__20220916152529.csv', encoding = 'cp949')
population = pd.read_csv('ses/도시지역_인구현황_시군구__20220916152707.csv', encoding = 'cp949')

idx_with_name = pd.read_csv('spatial/qgis_지도_idx_72.csv', encoding = 'utf-8')

In [8]:
hospital = hospital[['air_out_idx', '계']].copy()
hospital.rename(columns = {'계' : '요양기관 수'}, inplace = True)
hospital

,air_out_idx,요양기관 수
0,2,2821
1,3,960
2,6,941
3,27,883
4,35,735
...,...,...
67,137,262
68,247,731
69,31,321
70,143,203


In [9]:
money = money[['air_out_idx', '재정자립도(세입과목개편후)']].copy()
money

,air_out_idx,재정자립도(세입과목개편후)
0,196,50.0
1,197,58.6
2,168,40.1
3,117,34.5
4,32,27.4
...,...,...
67,31,48.6
68,143,30.7
69,234,24.7
70,150,20.7


In [10]:
population.drop(['소재지(시군구)별(1)', '소재지(시군구)별(2)'], axis = 1, inplace = True)
population.rename(columns = {'소계' : '전체인구', '도시인구(B)' : '도시인구(용도지역기준)', '비도시인구(C=A-B)': '비도시인구(용도지역기준)', 
                                    '도시인구(D)' : '도시인구(행정구역기준)', '농촌인구(E=A-D)' : '농촌인구(행정구역기준)', 
                                    '용도지역 인구기준(B/A*100)' : '도시지역인구비율(용도지역기준)', '행정구역 인구기준(D/A*100)' : '도시지역인구비율(행정구역기준)'}, inplace = True)

In [11]:
ses_2015 = pd.merge(hospital, money, on='air_out_idx')
ses_2015 = pd.merge(ses_2015, population, on = 'air_out_idx')
ses_2015

,air_out_idx,요양기관 수,재정자립도(세입과목개편후),전체인구,도시인구(용도지역기준),비도시인구(용도지역기준),도시인구(행정구역기준),농촌인구(행정구역기준),도시지역인구비율(용도지역기준),도시지역인구비율(행정구역기준)
0,2,2821,60.0,576495,576495,0,576495,0,100.00,100.00
1,3,960,29.2,458658,458658,0,458658,0,100.00,100.00
2,6,941,22.4,589074,589074,0,589074,0,100.00,100.00
3,27,883,21.6,509663,509663,0,509663,0,100.00,100.00
4,35,735,25.1,422092,422092,0,422092,0,100.00,100.00
...,...,...,...,...,...,...,...,...,...,...
67,137,262,33.6,180199,141507,38692,111239,68960,78.53,61.73
68,247,731,59.1,596525,507046,89479,515932,80593,85.00,86.49
69,31,321,48.6,312579,168349,144230,285608,26971,53.86,91.37
70,143,203,30.7,205184,149785,55399,168597,36587,73.00,82.17


In [12]:
# merge with main data

asthma_sudogwon_agg_2015SESadded = pd.merge(asthma, ses_2015, on='air_out_idx', how = 'left')
rhinitis_sudogwon_agg_2015SESadded = pd.merge(rhinitis, ses_2015, on='air_out_idx', how = 'left')
atopic_sudogwon_agg_2015SESadded = pd.merge(atopic, ses_2015, on='air_out_idx', how = 'left')

In [13]:
asthma_sudogwon_agg_2015SESadded.to_csv('outcome/asthma_sudogwon_agg_2015SESadded.csv', encoding = 'euc-kr', index = False)
rhinitis_sudogwon_agg_2015SESadded.to_csv('outcome/rhinitis_sudogwon_agg_2015SESadded.csv', encoding = 'euc-kr', index = False)
atopic_sudogwon_agg_2015SESadded.to_csv('outcome/atopic_sudogwon_agg_2015SESadded.csv', encoding = 'euc-kr', index = False)